<p style='text-align:center'>
PSY 394U <b>Methods for fMRI</b>, Fall 2018


<img style='width: 300px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Placebo_Left.png?raw=true' alt='brain blobs'/>

</p>

<p style='text-align:center; font-size:40px; margin-bottom: 30px;'><b> fMRI pre-processing </b></p>

<p style='text-align:center; font-size:18px; margin-bottom: 32px;'><b> November 5, 2018</b></p>

<hr style='height:5px;border:none' />

# Overview of pre-processing steps

We have seen in the previous classes how to normalize structural MRI data to the template space. Starting today, we will learn how to pre-process an fMRI data set so that it will be ready to be analyzed. There are several steps in fMRI pre-processing, and different software tools implement different steps at different stages of preprocessing. Here, my goal is to show some essential steps in pre-processing; whey these steps are needed, and how they can be implemented.

Here are steps of fMRI pre-processing. The order may vary in different software packages, but the idea behind these steps is the same.

  1. **Slice timing correction**: Corrects for the time differences slice acquisition
  2. **Motion correction / realignment**: Corrects for any spatial displacement during fMRI acquisition
  3. **Artifact detection**: Corrects for any physiological artifacts during acquisition
  4. **Co-registration**: Re-orients the image in the same space as the T1-weighted image
  5. **Normalization**: Applies the warping from T1-image normalization to transform fMRI data into the template space
  6. **Temporal filtering**: Eliminates low-frequency drift in BOLD fMRI time series
  7. **Spatial smoothing**: Smooth fMRI data with a Gaussian kernel

# Slice timing correction

## What is slice timing correction?

A typical 3D fMRI image is acquired as a series of 2D images, acquired in a certain order.

<img style='width: 400px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Preproc_SliceSeq.png?raw=true' alt='Sequential acquisition'/>

<p style='text-align:center; font-size:14px; margin-bottom: 32px; margin-top: 0px'> <i>Sequential (from top to bottom) </i> </p>


<img style='width: 400px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Preproc_SliceInter.png?raw=true' alt='Interleaved acquisition'/>

<p style='text-align:center; font-size:14px; margin-bottom: 32px; margin-top: 0px'> <i>Interleaved (odd slices, then even slices)</i> </p>

That means there is a delay between the slices acquired first and last, for example. That means these slices are capturing fMRI signal at the different time points. Slice timing correction corrects such differences by signal interpolation and phase shift.


## Slice timing correction or temporal derivatives?

If TR (repetition time - time required to acquire a single volume) is sufficiently short (<2 Sec), then you can correct slice timing differences by including temporal derivatives in your first-level regression model (to be covered later). 

## Slice timing correction with Nipype

Both SPM and FSL have slice timing correction. Here are code snippet examples implementing a slice timing correction for interleaved acquisition. ***They are not complete programs, but code snippets defining a node***.

### FSL

`<SliceTime_fsl.py>`

In [ ]:
from nipype.interfaces import fsl

sliceTime = Node(fsl.slicetimer(interleaved = True,  # interleaved
                                time_repetition = 2.5,  # TR in Sec
                                ),
                 name='sliceTime')

# To connect with other nodes, here are input and output parameters:
#
# input:   in_file
# output:  slice_time_corrected_file
#

It is possible to specify slice order or timing for custom cases (e.g., multi-band acquisition). For more details, please consult **`fsl.slicetimer.help()`**.

### SPM

`<SliceTime_spm.py>`

In [ ]:
from nipype.interfaces import spm

sliceTime = Node(spm.SliceTiming(num_slices = 32,  # number of slices
                                 time_repetition = 2.5,   # TR in sec
                                 time_acquisition = 2.5 - 2.5/32,  # acqu time
                                 slice_order = list(range(31,0,-2))+list(range(32,0,-2)),
                                                   # slice timing, interleaved
                                 ref_slice = 1     # reference slice
                                 )

# To connect with other nodes, here are input and output parameters:
#
# input:   in_files
# output:  timecorrected_files
#

# Motion correction / realignment